In [1]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import signal
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

import scipy.stats as scs
from sklearn.model_selection import train_test_split

plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df = pd.read_csv('data/mar2019-feb2020_top1.csv')
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1, inplace = True )

df.shape # (1048575, 15)
# df.head()
df.tail(1)

(104211, 26)

,Day,bikeid,birth year,dates,day of year,end station id,end station latitude,end station longitude,end station name,gender,...,starttime,stoptime,tripduration,usertype,week,weekday,start time,stop time,Trip in Minutes,age
104210,18,18317,1955,2019-04-18,108,540.0,40.743116,-73.982154,Lexington Ave & E 29 St,1,...,4/18/19 9:19,4/18/19 9:27,433,Subscriber,16,3,9:19,9:27,7.0,64


In [4]:
#df_with_dateindex = df.set_index('dates')
# df_with_dateindex.shape
# Not null value of y 
# Y_index = y[~np.isnan(y)]


# Linear regresion to predict missing values 


In [6]:
lr_feature = ['bikeid', 'dates',
       'day of year', 'end station name', 'gender', 'hour', 'month',
       'start station name', 'starttime', 'stoptime', 'tripduration',
       'usertype', 'week', 'weekday', 'starttime', 'stoptime', 'weekday'
      ]
lr_df = df[lr_feature]


In [7]:
# lr_df

In [8]:
lr_df = df.groupby('dates').count()['bikeid']
lr_df

dates
2019-03-01    345
2019-03-02     59
2019-03-03     51
2019-03-04    295
2019-03-05    347
             ... 
2020-02-23    146
2020-02-24    612
2020-02-25    372
2020-02-26    453
2020-02-27    202
Name: bikeid, Length: 256, dtype: int64

# Create new data frame with all 365 days dates as index 

In [11]:
# create index for a year 
index = pd.date_range(start='3/1/2019', end='2/28/2020')



In [72]:
counts =  pd.DataFrame(df.groupby('dates')['Day'].count())
counts.head()

,Day
dates,
2019-03-01,345
2019-03-02,59
2019-03-03,51
2019-03-04,295
2019-03-05,347


In [14]:
# dataframe with index dates 
# next, add counts of rides for that specific dates, other will be empty 
test_df = pd.DataFrame(index=index)

In [17]:
df_impute = pd.merge(test_df, counts, how='outer', left_index=True, right_index=True)

In [73]:
df_impute.head()

,bikeid,day count
2019-03-01,345.0,2130
2019-03-02,59.0,2131
2019-03-03,51.0,2132
2019-03-04,295.0,2133
2019-03-05,347.0,2134


In [74]:
# first day of city bike 2013-05-01
# find count of day from city bike start date 
from datetime import datetime
def days_between(d1, d2 = '2013-05-01'):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

num_days = []
for day in range(days_between(d1= '2019-03-01', d2 = '2013-05-01'), days_between(d1= '2020-02-29', d2 = '2013-05-01')):
    num_days.append(day)
len(num_days)

365

In [75]:
# count number of days based on 2013
df_impute['day count'] = num_days

In [77]:
df_impute.head()

,bikeid,day count
2019-03-01,345.0,2130
2019-03-02,59.0,2131
2019-03-03,51.0,2132
2019-03-04,295.0,2133
2019-03-05,347.0,2134


In [22]:
df_impute.rename(columns = {'Day':'bikeid'}, inplace = True) 

In [23]:
lr_df = df_impute.drop('day count', axis=1)

In [24]:
lr_df = lr_df.rename(columns={'count Target': 'bikeid'})

In [27]:
lr_df['day'] = lr_df.index.day
lr_df['month'] = lr_df.index.month
lr_df['weekday'] = lr_df.index.weekday
lr_df['time'] = list(range(len(lr_df)))

In [37]:

# train = lr_df.drop('dates')
# test = df['count Target']

In [38]:
# y_notnull = y[~np.isnan(y)]
# X_notnull = X[~np.isnan(y)]

In [39]:
not_null_index = lr_df.notnull().iloc[:, 0]

In [40]:
lr_df.columns

Index(['bikeid', 'day', 'month', 'weekday', 'time'], dtype='object')

In [42]:
merged = lr_df.copy()

In [43]:
dummies = pd.get_dummies(merged, columns = ['day', 'month', 'weekday'],drop_first=True)
# dummies

In [44]:
y = dummies.bikeid
X = dummies.drop('bikeid', axis = 1)

In [45]:
X = sm.add_constant(X)


# Scikit learn 






In [48]:
Y_index = y[~np.isnan(y)]
# Y_index

In [49]:
Y_index_to_predict = y[np.isnan(y)]
X_index_to_predict = X.loc[Y_index_to_predict.index]

In [50]:
X_with_vals = X.loc[Y_index.index]
Y_with_vals = y.loc[Y_index.index]

In [51]:
# Y_with_vals

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(X_with_vals, Y_with_vals)

In [53]:
# Linear regression; 

model_sk = LinearRegression()
model_sk.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [54]:
cross_val_score(model_sk, X_train, Y_train)

array([0.33190974, 0.3695117 , 0.44646527, 0.34618091, 0.50191063])

In [55]:
model_sk.score(X_test, Y_test)

0.4666830104427213

In [56]:
y_impute = model_sk.predict(X_index_to_predict)

In [57]:
# y_impute

In [58]:
to_impute_df = pd.DataFrame([Y_index_to_predict, y_impute ])

df_tom_make_y = pd.DataFrame(Y_index_to_predict)
df_tom_make_y['estimated_rides'] = y_impute

In [59]:
df_tom_make_y.head()


,bikeid,estimated_rides
2019-03-27,NaN,439.641586
2019-03-28,NaN,489.299070
2019-03-29,NaN,163.846281
2019-03-30,NaN,124.927539
2019-03-31,NaN,89.723755


In [60]:
df_impute

,bikeid,day count
2019-03-01,345.0,2130
2019-03-02,59.0,2131
2019-03-03,51.0,2132
2019-03-04,295.0,2133
2019-03-05,347.0,2134
...,...,...
2020-02-24,612.0,2490
2020-02-25,372.0,2491
2020-02-26,453.0,2492
2020-02-27,202.0,2493


# Add original data with missing values and predected data to make 365 days

In [61]:
given_df = df_impute.drop('day count', axis = 1)
given_df.rename(columns = {'bikeid':'rides'}, inplace = True) 

In [78]:
given_df.head()

,rides
2019-03-01,345.0
2019-03-02,59.0
2019-03-03,51.0
2019-03-04,295.0
2019-03-05,347.0


In [81]:
df_tom_make_y.head()

,bikeid,estimated_rides
2019-03-27,NaN,439.641586
2019-03-28,NaN,489.299070
2019-03-29,NaN,163.846281
2019-03-30,NaN,124.927539
2019-03-31,NaN,89.723755


In [64]:
predicted_impute = df_tom_make_y.drop('bikeid', axis = 1)
predicted_impute.rename(columns = {'estimated_rides':'rides'}, inplace = True) 
# predicted_impute

In [65]:
# combines with index same 
complete_df = given_df.combine_first(predicted_impute)

In [82]:
complete_df.head()

,rides
2019-03-01,345
2019-03-02,59
2019-03-03,51
2019-03-04,295
2019-03-05,347


In [83]:
complete_df['rides'] = complete_df["rides"].astype(int)
complete_df.head()

,rides
2019-03-01,345
2019-03-02,59
2019-03-03,51
2019-03-04,295
2019-03-05,347
